In [413]:
import pandas as pd 
import numpy as np 
from datetime import datetime
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn import preprocessing

In [414]:
# Reading in data
def parse(x):
    return datetime.strptime(x, '%m/%d/%Y')
march2020 = pd.read_csv('march2020clean.csv', sep = ',', date_parser = parse)


# Popping label and date columns
label = march2020.pop('closePrice')

date = march2020.pop('date')

# Dropping features that aren't highly correlated with our label
march2020 = march2020.drop(['openPrice', 'highPrice', 'lowPrice', 'totalSoybeanMealSupply',
         'totalSoybeanMealDemand', 'soybeanOilSupply', 
         'soybeanOilDemand', 'sunflowerSeedPrice', 'peanutsPrice', 'flaxseedPrice',
         'cottonseedOilPrice', 'sunflowerseedOilPrice', 'canolaOilPrice', 'peanutOilPrice',
        'Unnamed: 0'
        ], axis = 1) 

march2020.head()


,Month,canolaPrice,soybeanOilPrice,cornOilPrice,soybeanMealPrice,cottonseedmealPrice,sunflowerseedMealPrice,linseedMealPrice
0,November,17.2,33.43,34.46,313.52,228.75,165.0,205.0
1,November,17.2,33.43,34.46,313.52,228.75,165.0,205.0
2,November,17.2,33.43,34.46,313.52,228.75,165.0,205.0
3,November,17.2,33.43,34.46,313.52,228.75,165.0,205.0
4,November,17.2,33.43,34.46,313.52,228.75,165.0,205.0


In [415]:
months = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 
          'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10,
         'November': 11, 'December': 12}

Month = list(march2020['Month'])

def encode(l):
    encoded = []
    for m in Month:
        for key, value in months.items():
            if key == m:
                encoded.append(value)
    return encoded

Month = encode(Month)

march2020['Month'] = Month

month = march2020.pop('Month')

march2020.head()

,canolaPrice,soybeanOilPrice,cornOilPrice,soybeanMealPrice,cottonseedmealPrice,sunflowerseedMealPrice,linseedMealPrice
0,17.2,33.43,34.46,313.52,228.75,165.0,205.0
1,17.2,33.43,34.46,313.52,228.75,165.0,205.0
2,17.2,33.43,34.46,313.52,228.75,165.0,205.0
3,17.2,33.43,34.46,313.52,228.75,165.0,205.0
4,17.2,33.43,34.46,313.52,228.75,165.0,205.0


In [416]:
# Normalizing features
x = march2020.values 
scaler = preprocessing.StandardScaler()
x_scaled = scaler.fit_transform(x)
march2020 = pd.DataFrame(x_scaled)
#march2020['7'] = month 

march2020.head()


,0,1,2,3,4,5,6
0,0.594429,2.744969,2.750389,-0.625971,-0.766518,-0.431963,-1.329815
1,0.594429,2.744969,2.750389,-0.625971,-0.766518,-0.431963,-1.329815
2,0.594429,2.744969,2.750389,-0.625971,-0.766518,-0.431963,-1.329815
3,0.594429,2.744969,2.750389,-0.625971,-0.766518,-0.431963,-1.329815
4,0.594429,2.744969,2.750389,-0.625971,-0.766518,-0.431963,-1.329815


In [421]:
# Train test split
train_index_start = 0
train_index_end = 325 
test_index_start = 326
test_index_end = 407

train_features = march2020.iloc[train_index_start:train_index_end,:]
train_label = label[train_index_start:train_index_end]

test_features = march2020.iloc[test_index_start:test_index_end,:]
test_label = label[test_index_start:test_index_end]


# Creating sequence array
def splitSeq(df):
    seq_list = []
    for column in df:
        seq_list.append(list(df[column]))
        
resTrain = np.array(splitSeq(train_features))

resTest = np.array(splitSeq(test_features))

train_features = resTrain
test_features = resTest

train_label = np.array(train_label)
test_label = np.array(test_label)





array([965.  , 953.75, 950.75, 946.  , 942.  , 948.5 , 952.  , 948.75,
       956.5 , 953.25, 953.75, 955.  , 958.75, 949.75, 952.5 , 947.5 ,
       937.25, 937.  , 933.5 , 943.  , 948.25, 947.75, 954.75, 949.  ,
       948.25, 948.  , 950.5 , 944.5 , 943.75, 946.5 , 937.5 , 929.5 ,
       930.75, 927.5 , 915.  , 907.75, 912.  , 906.25, 899.25, 893.75,
       892.25, 883.5 , 883.5 , 874.5 , 873.25, 872.25, 859.25, 857.75,
       852.75, 879.25, 881.5 , 885.75, 868.  , 878.25, 868.5 , 875.25,
       869.5 , 876.  , 901.5 , 915.5 , 931.75, 921.75, 924.  , 927.75,
       917.75, 916.25, 905.5 , 909.  , 910.5 , 926.25, 934.  , 941.  ,
       954.5 , 955.75, 947.5 , 960.25, 947.  , 952.  , 944.25, 936.75,
       931.75])